In [3]:
library(readr)
library(dplyr)
library(tidyr)

In [4]:
### Reading in Data
fp <- '/pool001/mfzhao/'
weather <- foreach(year = 2010:2020, .combine = 'bind_rows') %dopar% 
    read_csv(str_c(fp, 'GHCN/', year, '.csv'), 
             col_names = c("STAID", "DATE", "type", "value", "X1", "X2", "SFLAG1", "Time"),
             col_types = cols(
                 STAID = col_character(),
                 DATE = col_double(),
                 type = col_character(),
                 value = col_double(),
                 X1 = col_character(),
                 X2 = col_character(),
                 SFLAG1 = col_character(),
                 Time = col_character()
             ))

In [5]:
weather %>%
    filter(type == 'TMAX' | type == 'PRCP') %>%
    select(STAID, DATE, type, value) %>%
    spread(key = type, value = value) -> weather

In [6]:
write_csv(weather, str_c(fp, 'PROCESSED_DATA/ghcn_weather.csv'))

write_csv(stations, str_c(fp, 'PROCESSED_DATA/ghcn_stations.csv'))

ERROR: Error in is.data.frame(x): object 'stations' not found


In [ ]:
fp <- '/pool001/mfzhao/'
stations <- read.fwf(str_c(fp, "GHCN/ghcnd-stations.txt"), widths = c(11,9,10,7,3,31,4,4,6), 
                     comment.char="", stringsAsFactors = F)
colnames(stations) <- c("STAID", "lat", "lng", "elev", "state", "STANAME", "GSN", "HCN/CRN", "WMO")

stations %>%
    select(STAID, STANAME, lat, lng) -> stations

In [ ]:
gec_to_iso <- read_csv(str_c(fp, 'geo_data/gec_to_iso.csv'))

stations %>%
    mutate(gec = str_sub(STAID, 1, 2)) %>%
    left_join(gec_to_iso) %>%
    select(STAID, STANAME, CN = iso2, lat, lng) -> stations

write_csv(stations, str_c(fp, 'PROCESSED_DATA/ghcn_stations.csv'))